In [1]:
import pandas as pd
import numpy as np

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array

## importing libraries

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import time
from sklearn import preprocessing
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics
import math
from numpy import mean
from numpy import std

from sklearn.metrics import mean_squared_error, r2_score


print('Done importing libraries')

Done importing libraries


In [2]:
df2 = pd.read_csv('data/allSummer_delhi.csv')
df2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df2.drop(["index"], axis=1, inplace=True)



Ozone = df2['O3']

OzoneP24 = Ozone.shift(-24)
OzoneP24 = OzoneP24.replace(np.nan, Ozone.mean())
df2['O3P24'] = OzoneP24

df2.info()
print(df2.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11040 entries, 0 to 11039
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   a            11040 non-null  int64  
 1   City         11040 non-null  object 
 2   Date         11040 non-null  object 
 3   Time         11040 non-null  object 
 4   PM2.5        11040 non-null  float64
 5   PM10         11040 non-null  float64
 6   NO           11040 non-null  float64
 7   NO2          11040 non-null  float64
 8   NOx          11040 non-null  float64
 9   NH3          11040 non-null  float64
 10  CO           11040 non-null  float64
 11  SO2          11040 non-null  float64
 12  O3           11040 non-null  float64
 13  Benzene      11040 non-null  float64
 14  Toluene      11040 non-null  float64
 15  Xylene       11040 non-null  float64
 16  O3P24        11040 non-null  float64
 17  Temperature  11040 non-null  float64
dtypes: float64(14), int64(1), object(3)
memory usa

In [3]:
## defining variables

#X = df2[['PM10', 'NO', 'NO2', 'CO', 'SO2', 'O3', 'Toluene', 'Temp']] # , 'Humid', Xylene
X = df2[['PM10', 'NO', 'NO2', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train1.shape[0]))
print("Size of testing dataset: {} rows".format(X_test1.shape[0]))

print('Done defining variables')

Size of training dataset: 9936 rows
Size of testing dataset: 1104 rows
Done defining variables


In [4]:
## scaling variables

'''importing libraries'''
from sklearn.preprocessing import RobustScaler

scaler = preprocessing.RobustScaler()
X_train_scaled1 = scaler.fit_transform(X_train1)
X_test_scaled1 = scaler.fit_transform(X_test1)

print('Done scaling :)')

Done scaling :)


In [5]:
## linear regression
t0 = time.time()

'''importing library'''
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

'''create linear regressor object'''
lnRegressor = LinearRegression()

'''fitting regressor'''
lnRegressor.fit(X_train1, y_train)

# cv stuff

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

cvln_r2scores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvln_r2scores)
cvln_rmsescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvln_rmsescores)
cvln_maescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvln_maescores)

# calculating r^2 adj score

cvln_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvln_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvln_adj.append(adj_r2)    

cvln_r2score = round((np.mean(cvln_r2scores)), 4)
cvln_adjscore = round((np.mean(cvln_adj)), 4)
cvln_rmsescore = (round(abs(np.mean(cvln_rmsescores)),2))
cvln_maescore = (round(abs(np.mean(cvln_maescores)),2))


print(cvln_r2score, cvln_adjscore, cvln_rmsescore, cvln_maescore)

t1 = time.time()
total = t1-t0

print("done training linear regressor | time taken: %f seconds" %total)
yPredln = lnRegressor.predict(X_test1)
print('The average R2 value for linear regressor is :', cvln_r2score)

[0.48490162 0.44792192 0.49090641 0.44574002 0.48385559 0.51029229
 0.4369715  0.43755487 0.39463913 0.43967979]
[-25.22482848 -23.57292149 -25.82692541 -24.68474595 -26.36544386
 -23.8953288  -25.30173488 -24.59353197 -27.56473267 -25.24792608]
[-17.85744289 -16.78377211 -17.3604169  -16.91563638 -17.67876081
 -16.32898955 -17.74128477 -17.17535554 -18.07379378 -17.12343278]
0.4572 0.4567 25.23 17.3
done training linear regressor | time taken: 2.119817 seconds
The average R2 value for linear regressor is : 0.4572


In [6]:
## kneighbors regression
print("STARTING: KN")

t0 = time.time()

'''importing library'''
from sklearn.neighbors import KNeighborsRegressor

'''create regressor object'''
knRegressor = KNeighborsRegressor(n_neighbors = 4, metric = 'minkowski', p = 1)

'''fitting regressor'''
knRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvkn_r2scores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvkn_r2scores)
cvkn_rmsescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvkn_rmsescores)
cvkn_maescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvkn_maescores)

# calculating r^2 adj score

cvkn_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvkn_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvkn_adj.append(adj_r2)    

cvkn_r2score = round((np.mean(cvkn_r2scores)), 4)
cvkn_adjscore = round((np.mean(cvkn_adj)), 4)
cvkn_rmsescore = (round(abs(np.mean(cvkn_rmsescores)),2))
cvkn_maescore = (round(abs(np.mean(cvkn_maescores)),2))

print(cvkn_r2score, cvkn_adjscore, cvkn_rmsescore, cvkn_maescore)


t1 = time.time()
total = t1-t0

print("done training knn regressor | time taken: %f seconds" %total)

yPredkn = knRegressor.predict(X_test_scaled1)
print('The average R2 value for KNN Regressor is :', cvln_r2score)

STARTING: KN
[0.60300259 0.55132032 0.62418919 0.53896117 0.5715894  0.54931696
 0.4542407  0.55524927 0.56938301 0.5012207 ]
[-22.14506538 -21.25109721 -22.1900627  -22.51337746 -24.02036338
 -22.9234589  -24.91068585 -21.86951535 -23.24835288 -23.82109804]
[-14.73060342 -13.72639261 -14.40131868 -14.57430409 -15.34011115
 -14.60066656 -16.24264435 -14.68711294 -14.77211365 -15.14591872]
0.5518 0.5514 22.89 14.82
done training knn regressor | time taken: 3.320569 seconds
The average R2 value for KNN Regressor is : 0.4572


In [7]:
## svr regressor
print("STARTING: SVR")

t0 = time.time()

'''importing library'''
from sklearn.svm import SVR

'''create regressor object'''
svrRegressor = SVR(C = 10, gamma = 0.1, kernel= 'rbf')

'''fitting regressor'''
svrRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvsvr_r2scores = cross_val_score(svrRegressor, X_train_scaled1, y_train, cv = 10, scoring = 'r2')
print(cvsvr_r2scores)
cvsvr_rmsescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvsvr_rmsescores)
cvsvr_maescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvsvr_maescores)


# calculating r^2 adj score

cvsvr_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvsvr_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvsvr_adj.append(adj_r2)

cvsvr_r2score = round((np.mean(cvsvr_r2scores)), 4)
cvsvr_adjscore = round((np.mean(cvsvr_adj)), 4)
cvsvr_rmsescore = (round(abs(np.mean(cvsvr_rmsescores)),2))
cvsvr_maescore = (round(abs(np.mean(cvsvr_maescores)),2))

print(cvsvr_r2score, cvsvr_adjscore, cvsvr_rmsescore, cvsvr_maescore)

t1 = time.time()
total = t1-t0

print("done training svr regressor | time taken: %f seconds" %total)

yPredsvr = svrRegressor.predict(X_test_scaled1)
print('The average R2 value for SVR Regressor is :', cvsvr_r2score)

STARTING: SVR
[0.52130386 0.50268041 0.52428374 0.54212199 0.49274679 0.51898355
 0.48372326 0.49800025 0.46865569 0.51252559]
[-36.11578311 -32.42793304 -37.26960058 -34.27358029 -37.71861817
 -34.83250286 -34.59114294 -33.19716698 -36.96097824 -34.6811009 ]
[-27.28828207 -25.40659848 -27.65180403 -27.14031442 -27.93863292
 -25.47157372 -25.89763678 -25.0692909  -26.96935717 -26.16344092]
0.5065 0.506 35.21 26.5
done training svr regressor | time taken: 480.241063 seconds
The average R2 value for SVR Regressor is : 0.5065


In [8]:
## defining variables WITH MULTICOLLINEARITY: random forest, decision tree, maybe adaboost, and xgboost

X = df2[['PM10', 'NO', 'NO2', 'NOx', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train.shape[0]))
print("Size of testing dataset: {} rows".format(X_test.shape[0]))

print('Done defining NEW variables')

Size of training dataset: 9936 rows
Size of testing dataset: 1104 rows
Done defining NEW variables


In [9]:
## random forest regressor
print("STARTING: rf")
t0 = time.time()

'''importing library'''
from sklearn.ensemble import RandomForestRegressor

'''create regressor object'''
rfRegressor = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=250) 

'''fitting regressor'''
rfRegressor.fit(X_train, y_train)

# cv stuff

cvrf_r2scores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'r2') # x_train was scaled for some reason, check if it changes things?
print(cvrf_r2scores)
cvrf_rmsescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvrf_rmsescores)
cvrf_maescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvrf_maescores)


# calculating r^2 adj score

cvrf_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvrf_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvrf_adj.append(adj_r2)

cvrf_r2score = round((np.mean(cvrf_r2scores)), 4)
cvrf_adjscore = round((np.mean(cvrf_adj)), 4)
cvrf_rmsescore = (round(abs(np.mean(cvrf_rmsescores)),2))
cvrf_maescore = (round(abs(np.mean(cvrf_maescores)),2))

print(cvrf_r2score, cvrf_adjscore, cvrf_rmsescore, cvrf_maescore)

print(cvrf_r2score, cvrf_adjscore)

t1 = time.time()
total = t1-t0

print("done training random forest regressor | time taken: %f seconds" %total)
yPredrf = rfRegressor.predict(X_test)
print('The average R2 value for Random Forest Regressor is :', cvrf_r2score)

## printing feature importances

importances = rfRegressor.feature_importances_

sorted_indices = np.argsort(importances)[::-1]

print('feature importances:::')
print(sorted_indices)

STARTING: rf
[0.66252079 0.58824934 0.61759453 0.60086668 0.59744577 0.58831634
 0.57935197 0.56732101 0.60311821 0.57616741]
[-20.41769177 -20.35777513 -22.38390952 -20.94743699 -23.28421866
 -21.90919183 -21.86978641 -21.5706747  -22.31912677 -21.95860466]
[-14.05905029 -13.5137012  -14.36225306 -13.97592438 -15.00766471
 -14.12888391 -14.89528028 -14.42898278 -14.79782039 -14.09355311]
0.5981 0.5976 21.7 14.33
0.5981 0.5976
done training random forest regressor | time taken: 1555.129646 seconds
The average R2 value for Random Forest Regressor is : 0.5981
feature importances:::
[ 7  1  6  0 10  4  2  5  3  8  9]


In [10]:
## decision tree regressor
print('STARTING: dt')
t0 = time.time()

'''importing library'''
from sklearn.tree import DecisionTreeRegressor

'''create regressor object'''
dtRegressor = DecisionTreeRegressor(random_state=0, max_depth = 6)

'''fitting regressor'''
dtRegressor.fit(X_train,y_train)

# cv stuff

cvdt_r2scores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvdt_r2scores)
cvdt_rmsescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvdt_rmsescores)
cvdt_maescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvdt_maescores)


# calculating r^2 adj score

cvdt_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvdt_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvdt_adj.append(adj_r2)

cvdt_r2score = round((np.mean(cvdt_r2scores)), 4)
cvdt_adjscore = round((np.mean(cvdt_adj)), 4)
cvdt_rmsescore = (round(abs(np.mean(cvdt_rmsescores)),2))
cvdt_maescore = (round(abs(np.mean(cvdt_maescores)),2))

print(cvdt_r2score, cvdt_adjscore, cvdt_rmsescore, cvdt_maescore)

t1 = time.time()
total = t1-t0

print("done training decision tree regressor | time taken: %f seconds" %total)

yPreddt = dtRegressor.predict(X_test)
print('The average R2 value for Decision Tree Regressor is :', cvdt_r2score)

STARTING: dt
[0.53797857 0.38035309 0.5481056  0.43967375 0.43947542 0.45476987
 0.46301959 0.45319461 0.47369306 0.43015987]
[-23.88989223 -24.97383866 -24.33281602 -24.8194631  -27.47557585
 -25.21357708 -24.70952147 -24.24918897 -25.70195513 -25.46150561]
[-16.53834131 -16.53774107 -16.32391555 -16.33094467 -17.6324303
 -16.56556429 -17.19028729 -16.5344801  -17.09459559 -16.66301576]
0.462 0.4614 25.08 16.74
done training decision tree regressor | time taken: 3.202802 seconds
The average R2 value for Decision Tree Regressor is : 0.462


In [11]:
## adaboost regressor
print('STARTING: adaboost regressor')
t0 = time.time()

'''importing library'''
from sklearn.ensemble import AdaBoostRegressor

'''create regressor object'''
adaRegressor = AdaBoostRegressor(random_state=0, learning_rate = 0.1, n_estimators=100)

'''fitting regressor'''
adaRegressor.fit(X_train, y_train)

# cv stuff

cvada_r2scores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvada_r2scores)
cvada_rmsescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvada_rmsescores)
cvada_maescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvada_maescores)

# calculating r^2 adj score

cvada_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvada_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvada_adj.append(adj_r2)

cvada_r2score = round((np.mean(cvada_r2scores)), 4)
cvada_adjscore = round((np.mean(cvada_adj)), 4)
cvada_rmsescore = (round(abs(np.mean(cvada_rmsescores)),2))
cvada_maescore = (round(abs(np.mean(cvada_maescores)),2))

print(cvada_r2score, cvada_adjscore, cvada_rmsescore, cvada_maescore)

t1 = time.time()
total = t1-t0

print("done training Adaboost regressor | time taken: %f seconds" %total)

yPredada = adaRegressor.predict(X_test)
print('The average R2 value for Adaboost Regressor is :', cvada_r2score)

STARTING: adaboost regressor
[0.47512286 0.40343334 0.48486353 0.44175143 0.44969951 0.44952208
 0.40333685 0.39105498 0.41824714 0.40958738]
[-25.46314001 -24.50431968 -25.97975449 -24.77340535 -27.22384237
 -25.33462573 -26.04652084 -25.58997527 -27.02189954 -25.91703896]
[-19.28973923 -18.79602822 -19.16079145 -18.6229973  -19.91062304
 -18.61456454 -19.86389836 -19.45982146 -20.11898774 -18.82064131]
0.4327 0.432 25.79 19.27
done training Adaboost regressor | time taken: 100.870426 seconds
The average R2 value for Adaboost Regressor is : 0.4327


In [12]:
## xgboost regressor
print('STARTING: xgboost regressor')
t0 = time.time()

'''importing library'''
import xgboost as xgb

'''create regressor object'''
xgbRegressor = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=300, verbosity = 0, random_state = 0, silent = True)

'''fitting regressor'''
xgbRegressor.fit(X_train, y_train)

# cv stuff

cvxgb_r2scores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvxgb_r2scores)
cvxgb_rmsescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvxgb_rmsescores)
cvxgb_maescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvxgb_maescores)


# calculating r^2 adj score

cvxgb_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvxgb_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvxgb_adj.append(adj_r2)

cvxgb_r2score = round((np.mean(cvxgb_r2scores)), 4)
cvxgb_adjscore = round((np.mean(cvxgb_adj)), 4)
cvxgb_rmsescore = (round(abs(np.mean(cvxgb_rmsescores)),2))
cvxgb_maescore = (round(abs(np.mean(cvxgb_maescores)),2))

print(cvxgb_r2score, cvxgb_adjscore, cvxgb_rmsescore, cvxgb_maescore)

t1 = time.time()
total = t1-t0

print("done training xgboost regressor | time taken: %f seconds" %total)

yPredxgb = xgbRegressor.predict(X_test)
print('The average R2 value for xgboost Regressor is :', cvxgb_r2score)

STARTING: xgboost regressor
[0.66793635 0.56540861 0.63420548 0.59293641 0.59486861 0.5969807
 0.57378597 0.59033701 0.62127423 0.5574885 ]
[-20.25320675 -20.91480015 -21.89235534 -21.1545129  -23.35863274
 -21.67741365 -22.01400099 -20.98911806 -21.80263766 -22.43726142]
[-13.98210006 -13.78274599 -14.11856033 -13.88828582 -14.86149627
 -13.80244901 -14.71161395 -13.85733942 -14.60028918 -14.07876801]
0.5995 0.5991 21.65 14.17
done training xgboost regressor | time taken: 358.952746 seconds
The average R2 value for xgboost Regressor is : 0.5995


In [13]:
##### printing cross-validations scores
# printing cross-validaion r^2 scores

print('The R^2 value for Linear Regression is         :', cvln_r2score)
print('The R^2 value for KNN Regressor is             :', cvkn_r2score)
print('The R^2 value for SVM Regressor is             :', cvsvr_r2score)
print('The R^2 value for Random Forests Regressor is  :', cvrf_r2score)
print('The R^2 value for Decision Tree Regressor is   :', cvdt_r2score)
print('The R^2 value for AdaBoost Regressor is        :', cvada_r2score)
print('The R^2 value for XGBoost Regressor is         :', cvxgb_r2score)

# printing cross-validation adjusted r^2 scores

print('The Adj. R^2 value for Linear Regression is        :', cvln_adjscore)
print('The Adj. R^2 value for KNN Regressor is            :', cvkn_adjscore)
print('The Adj. R^2 value for SVM Regressor is            :', cvsvr_adjscore)
print('The Adj. R^2 value for Random Forests Regressor is :', cvrf_adjscore)
print('The Adj. R^2 value for Decision Tree Regressor is  :', cvdt_adjscore)
print('The Adj. R^2 value for AdaBoost Regressor is       :', cvada_adjscore)
print('The Adj. R^2 value for XGBoost Regressor is        :', cvxgb_adjscore)

# printing cross-validation rmse scores

print('The RSME value for Linear Regression is         :', cvln_rmsescore)
print('The RSME value for KNN Regressor is             :', cvkn_rmsescore)
print('The RSME value for SVM Regressor is             :', cvsvr_rmsescore)
print('The RSME value for Random Forests Regressor is  :', cvrf_rmsescore)
print('The RSME value for Decision Tree Regressor is   :', cvdt_rmsescore)
print('The RSME value for AdaBoost Regressor is        :', cvada_rmsescore)
print('The RSME value for XGBoost Regressor is         :', cvxgb_rmsescore)

# printing cross-validation mae scores

print('The MAE value for Linear Regression is        :', cvln_maescore)
print('The MAE value for KNN Regressor is            :', cvkn_maescore)
print('The MAE value for SVM Regressor is            :', cvsvr_maescore)
print('The MAE value for Random Forests Regressor is :', cvrf_maescore)
print('The MAE value for Decision Tree Regressor is  :', cvdt_maescore)
print('The MAE value for AdaBoost Regressor is       :', cvada_maescore)
print('The MAE value for XGBoost Regressor is        :', cvxgb_maescore)

The R^2 value for Linear Regression is         : 0.4572
The R^2 value for KNN Regressor is             : 0.5518
The R^2 value for SVM Regressor is             : 0.5065
The R^2 value for Random Forests Regressor is  : 0.5981
The R^2 value for Decision Tree Regressor is   : 0.462
The R^2 value for AdaBoost Regressor is        : 0.4327
The R^2 value for XGBoost Regressor is         : 0.5995
The Adj. R^2 value for Linear Regression is        : 0.4567
The Adj. R^2 value for KNN Regressor is            : 0.5514
The Adj. R^2 value for SVM Regressor is            : 0.506
The Adj. R^2 value for Random Forests Regressor is : 0.5976
The Adj. R^2 value for Decision Tree Regressor is  : 0.4614
The Adj. R^2 value for AdaBoost Regressor is       : 0.432
The Adj. R^2 value for XGBoost Regressor is        : 0.5991
The RSME value for Linear Regression is         : 25.23
The RSME value for KNN Regressor is             : 22.89
The RSME value for SVM Regressor is             : 35.21
The RSME value for Rand